Analisis del dataset de SF BAY AREA BIKE SHARE en pandas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
trips = pd.read_csv('../input/trip.csv')

In [ ]:
weather = pd.read_csv('../input/weather.csv')

In [ ]:
station = pd.read_csv('../input/station.csv')

In [ ]:
trips.head(3)

# Depuracion de datos

In [ ]:
#separo las fechas de las horas
fechaYhora = trips['start_date'].str.split(' ')
fecha = fechaYhora.map(lambda x: x[0])
hora = fechaYhora.map(lambda x: x[1])
trips = trips.assign(date_start = fecha, time_start = hora)

In [ ]:
#IDEM pero con la fecha final
fechaYhora = trips['end_date'].str.split(' ')
fecha = fechaYhora.map(lambda x: x[0])
hora = fechaYhora.map(lambda x: x[1])
trips = trips.assign(date_end = fecha, time_end = hora)

In [ ]:
#le cambio el formato a las fechas
import calendar
trips['date_start'] = pd.to_datetime(trips['date_start'])
trips['date_end'] = pd.to_datetime(trips['date_end'])

In [ ]:
#Creacion de columna por nombre de dia
trips['weekday'] = trips['date_start'].map(lambda x: x.weekday_name)

In [ ]:
#Creacion de columna por hora
trips['hour_start'] = pd.to_datetime(trips['time_start'], format = '%H:%M', errors = 'coerce').dt.hour

## Cantidad de alquileres de bicicletas/duracion de viajes en relacion a las horas

In [ ]:
horaDuracion = trips.loc[:,['duration','hour_start']].groupby('hour_start')['duration'].mean().sort_index()

In [ ]:
plt.figure()
plt.title('Promedio de duraciones de viajes por hora')
horaDuracion.plot(kind='bar',figsize=(15,5))
plt.xlabel('Hora inicial')
plt.ylabel('Promedio de duracion');

In [ ]:
#Se ve en el grafico anterior que el promedio de las 3 am es mucho mayor que los anteriores, verifiquemos las cantidades en total
# Si hay un pico
#Plots sobre la Cantidad de alquileres por hora
plt.figure()
plt.title('Cantidad de Alquileres de bicicletas por hora', color = 'black')
trips['hour_start'].value_counts().sort_index().plot(kind = 'bar', figsize = (15,5))
plt.xlabel('Hora')
plt.ylabel('Cantidad bicicletas alquiladas');
#Se puede apreciar que hay muy pocos viajes a las 3 am, tanto asi afecta en el promedio gradualmente 

In [ ]:
plt.figure()
plt.title('Total de duraciones de viajes por hora')
trips.loc[:,['duration','hour_start']].groupby('hour_start')['duration'].sum().sort_index().plot(kind='bar',figsize=(15,5))
plt.xlabel('Hora inicial')
plt.ylabel('Cantidad total de duracion');

In [ ]:
# Cantidad de viajes segun dia, ordenados por dia de la semana
diasDeSemana = trips['weekday'].value_counts()
diasDeSemana = diasDeSemana.reindex(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
diasDeSemana

In [ ]:
#cantidad de alquileres por dia
plt.figure()
plt.title('Cantidad de Alquileres de bicicletas por dia', color = 'black')
diasDeSemana.plot(kind = 'bar', figsize = (12,5))
plt.ylabel('Cantidad de bicicletas alquiladas')
plt.xlabel('Dia');

# Analisis atributo bike_id

In [ ]:
top = trips.loc[:,['bike_id','duration']].groupby('bike_id').agg(['count','mean'])
top_ten = top.sort_values(('duration','count'),ascending=False).reset_index()

In [ ]:
top_ten.head(3)
#el bike_id 392 es el que mas viajes tiene

In [ ]:
trips.loc[trips.bike_id == 392,:].sort_values('duration',ascending=False)[0:5]

In [ ]:
#1 bici con mas viajes
plt.title('Histograma de duracion de la bicicleta con mas viajes (id 392)')
trips.loc[trips.bike_id == 392,:].loc[trips.duration <=4000]['duration'].hist(bins=100,figsize=(12,7))
plt.xlabel('Duracion')
plt.ylabel('Cantidad');

In [ ]:
# 2° bici con mas viajes
plt.title('Histograma de duracion de la bicicleta con mas viajes (id 489)')
trips.loc[trips.bike_id == 489,:].loc[trips.duration <=4000]['duration'].hist(bins=100,figsize=(12,7))
plt.xlabel('Duracion')
plt.ylabel('Cantidad');

In [ ]:
# 3° bici con mas viajes
plt.title('Histograma de duracion de la bicicleta con mas viajes (id 558)')
trips.loc[trips.bike_id == 558,:].loc[trips.duration <=4000]['duration'].hist(bins=100,figsize=(12,7))
plt.xlabel('Duracion')
plt.ylabel('Cantidad');

In [ ]:
plt.title('Histograma de duracion de las tres bicicletas con mas viajes')
trips.loc[trips.bike_id == 392,:].loc[trips.duration <=4000]['duration'].hist(bins=100,alpha=0.3,figsize=(12,7),color='yellow',label='id 392')
trips.loc[trips.bike_id == 558,:].loc[trips.duration <=4000]['duration'].hist(bins=100,alpha=0.3,figsize=(12,7),color='blue',label='id 558')
trips.loc[trips.bike_id == 489,:].loc[trips.duration <=4000]['duration'].hist(bins=100,alpha=0.3,figsize=(12,7),color='green',label='id 482')
plt.xlabel('Duracion')
plt.ylabel('Cantidad')
plt.legend();

In [ ]:
# Separo meses por nombre
trips['Month'] = trips['date_start'].map(lambda x: x.strftime('%B'))

In [ ]:
# Me fijo cada mes por los tres anios 2013,2014,2015
mesPor2013 = trips.loc[trips.start_date.str.contains('2013'),['Month']]
mesPor2013 = mesPor2013['Month'].value_counts()
mesPor2014 = trips.loc[trips.start_date.str.contains('2014'),['Month']]
mesPor2014 = mesPor2014['Month'].value_counts()
mesPor2015 = trips.loc[trips.start_date.str.contains('2015'),['Month']]
mesPor2015 = mesPor2015['Month'].value_counts()

In [ ]:
#Arreglo los indices
mesPor2014 = mesPor2014\
    .reindex(['January','February','March','April','May','June','July','August','September','October','November','December'])
mesPor2013 = mesPor2013.reindex\
(['January','February','March','April','May','June','July','August','September','October','November','December'],fill_value=0)
mesPor2015 = mesPor2015.reindex(\
['January','February','March','April','May','June','July','August','September','October','November','December'],fill_value=0)

In [ ]:
plt.figure()
plt.title('Cantidad de viajes segun anio y meses')
mesPor2013.plot('bar',position=1,width=0.1,label='2013',figsize=(15,5))
mesPor2014.plot('bar',position=0,color='green',width=0.13,label='2014',figsize=(15,5))
mesPor2015.plot('bar',position=2,color='black',width=0.13,label='2015',figsize=(15,5))
plt.legend()
plt.xlabel('Mes')
plt.ylabel('Cantidad de viajes');

# Subscriber y Customers

In [ ]:
numSuscripcion = trips['subscription_type'].value_counts()
numSuscripcion

In [ ]:
plt.title("Cantidad de alquileres de los clientes vs no clientes")
numSuscripcion.plot(kind = 'bar', figsize = (10,6));

In [ ]:
trips.head(3)

In [ ]:
trips['month_start'] = trips['date_start'].map(lambda x: x.month)

In [ ]:
#Cambio de nombre de las columnas anteriores
trips = trips.rename(columns={'weekday': 'day_start','Month' : 'month_start_name'})

In [ ]:
trips.sort_values('duration', ascending=False)[0:3]

In [ ]:
#Subscriber
tripSub = trips.loc[trips.subscription_type.str.contains('Subscriber')]\
               .loc[trips.duration >= 10,:].loc[trips.duration <=86400] #un dia

In [ ]:
plt.title('Tiempo promedio que viaja un Subscriptor para cada hora',color='black')
tripSub.groupby('hour_start').mean()['duration'].plot(figsize=(10,6));
plt.ylabel('Tiempo promedio de viaje')
plt.xlabel('hora');

In [ ]:
tripCus = trips.loc[trips.subscription_type.str.contains('Customer')]\
               .loc[trips.duration >= 60,:].loc[trips.duration <=86400] #un dia

In [ ]:
plt.title('Tiempo promedio que viaja un Customer para cada hora',color='black')
tripCus.groupby('month_start').mean()['duration'].plot(figsize=(10,6));
plt.ylabel('Tiempo promedio de viaje')
plt.xlabel('hora');

# Cantidad de viajes entre estaciones

In [ ]:
#cambio zip code a numerico
trips.zip_code = trips.zip_code.str.replace('nil','')
trips.zip_code = pd.to_numeric(trips.zip_code,errors='coerce')
trips.dtypes

In [ ]:
##Cantidad de Viajes por ZIP CODE
trips.groupby('zip_code').count().sort_values('start_station_name',ascending=False).head(10)

In [ ]:
#Localizo las estaciones que empezaron en una estacion y terminaron en otra 
# Saco las que tienen pocas (ecuacion mas peligrosa) los que tienen mas de 500 viajes
#Saco el top 10 de las duraciones por nombre de estacion inicial
viajesDistintasEstaciones = trips.loc[trips.start_station_name != trips.end_station_name,['start_station_name','duration']]
agrupadoViajesDistintos = viajesDistintasEstaciones.groupby('start_station_name').agg(['count','mean'])
topDuracionViajesEntreEstaciones = agrupadoViajesDistintos[agrupadoViajesDistintos['duration','count'] > 500 ]\
    ['duration','mean'].sort_values(ascending=False)
topDuracionViajesEntreEstaciones

In [ ]:
#Hago un plot de lo anterior
plt.figure()
plt.title('Top 10 de las duraciones entre viajes de distintas estaciones')
topDuracionViajesEntreEstaciones = topDuracionViajesEntreEstaciones.head(10)
topDuracionViajesEntreEstaciones.plot('bar',figsize= (16, 6))
plt.xlabel('Estacion inicial')
plt.ylabel('Promedio de duracion de todos los viajes');

In [ ]:
##Cantidad de viajes de una estacion a la misma
plt.figure('Top 10 cantidad de viajes desde una estacion a la misma')
trips.loc[trips.start_station_name == trips.end_station_name,'start_station_name'].value_counts().head(10)\
    .plot('bar',figsize=(14,6))
plt.ylabel('Cantidad')
plt.xlabel('Estacion')

# Cantidad de viajes entre distintas estaciones por ej: NY -> LA o LA -> NY

In [ ]:
#Cantidad de viajes entre distintas estaciones por  ej: NY -> LA o LA -> NY
entreEstacionesDistintas = trips.loc\
    [trips.start_station_name != trips.end_station_name,['start_station_name','end_station_name']]

In [ ]:
# creo una nueva columna iniciandizandolas en 1
entreEstacionesDistintas['contador'] = 1

In [ ]:
## Cambio de lugar las estaciones de comienzo y las finaless
entreEstacionesDistintasFinales = pd.DataFrame(entreEstacionesDistintas)
entreEstacionesDistintasFinales.rename(columns=\
                            {'start_station_name':'end_station_name','end_station_name':'start_station_name'},inplace=True)

In [ ]:
#Creo un contador de estaciones iniciales distintas de por ej: LA -> NY 
contadorEstacionesDistintasIniciales = entreEstacionesDistintas.groupby(['start_station_name','end_station_name']).sum()\
        .reset_index().sort_values('contador',ascending=False)

In [ ]:
#Creo un contador de estaciones finales distintas de por ej: NY -> LA 
contadorEstacionesDistintasFinales = entreEstacionesDistintasFinales.groupby(['start_station_name','end_station_name'])\
                            .count().reset_index().sort_values('contador',ascending=False)
contadorEstacionesDistintasFinales

# Merge

In [ ]:
# Los mergeo entre estaciones iniciales y finales que ya fueron sumados sus contadores, de forma INNER para que
# mezcle solo los que coincidan y no me haga datos repetidos
contadorEstacionesDistintasTotales = pd.merge(contadorEstacionesDistintasIniciales,contadorEstacionesDistintasFinales,\
                                             on=['start_station_name','end_station_name'],how='inner')

In [ ]:
# Defino una funcion para sumar los dos contadores
def sumarContadores(x):
    return x[2] + x[3]

In [ ]:
# aplico la funcion en el dataframe para sumar las columnas de contadores asi devuelve una serie con los resultados
sumaDeContadores = contadorEstacionesDistintasTotales.apply(sumarContadores,axis=1)
contadorEstacionesDistintasTotales['contador'] = sumaDeContadores

In [ ]:
contadorEstacionesDistintasTotales.head(10)

In [ ]:
# saco las columnas que estan de mas
del contadorEstacionesDistintasTotales['contador_x']
del contadorEstacionesDistintasTotales['contador_y']

In [ ]:
#Lo ordeno
contadorEstacionesDistintasTotales = contadorEstacionesDistintasTotales.sort_values('contador',ascending=False)

In [ ]:
## Recorro el dataframe para borrarles los que coinciden iniciales y finales
# Primero agrupo
grouped = contadorEstacionesDistintasTotales.groupby(['start_station_name','end_station_name'])
# Me creo otro dataframe
aBorrar = pd.DataFrame(contadorEstacionesDistintasTotales)
#Creo una lista vacia para ir agregando las estaciones ya cargadas
lista = []

for estacion,grupo in grouped:
#    print(estacion)
# Agarro la primera estacion y la "doy vuelta"
    estacionActual = (estacion[1],estacion[0])
# Si la estacion Actual (la que esta dada vuelta ) no esta en la lista la cargo, despues verifico si esta en el data frame
# la estacion a verificar y me quedo con los que no son iguales (Saco solo la que esta dada vuelta)
    if not(estacionActual) in lista:
        #Agrego a la lista
        lista.append(estacion)
        #Hago un loc para quedarme con la parte del dataframe que necesito
        aBorrar = aBorrar.loc[(estacion[0] != aBorrar['end_station_name'] ) | \
                    ((estacion[1] != aBorrar['start_station_name'])),:]
        

In [ ]:
# Saco el top 10 de la cantidad de viajes entre distintas estaciones 
top10 = aBorrar.head(10)
top10

In [ ]:
plt.figure()
plt.title(' Top 10 Cantidad de viajes entre distintas estaciones')
top10.groupby(['start_station_name','end_station_name'])['contador'].sum().plot(kind='bar',figsize=(9,6))
plt.ylabel('Cantidad de viajes')
plt.xlabel('Viajes entre estaciones');

# Cambio de temperatures en el weather de °F a °C

In [ ]:
#Cambio de grados Fahrenheit a grados Celcius
weather['max_temperature_f'] = weather['max_temperature_f'].map(lambda x: round((x-32)/1.8,1))
weather['min_temperature_f'] = weather['min_temperature_f'].map(lambda x: round((x-32)/1.8,1))
weather['mean_temperature_f'] = weather['mean_temperature_f'].map(lambda x: round((x-32)/1.8,1))

In [ ]:
## Renombro las columnas
weather.rename(columns={'max_temperature_f' : 'max_temperature',\
                        'min_temperature_f' : 'min_temperature','mean_temperature_f' : 'mean_temperature'},inplace=True)

In [ ]:
weather['date'] = pd.to_datetime(weather['date'])
weather['year'] = weather['date'].map(lambda x: x.year)
weather['month'] = weather['date'].map(lambda x: x.month)

In [ ]:
weather.head(3)

## Relacion Zip Code de weather con las cuidades del station y mergeo con trip

In [ ]:
station['city'].value_counts()

In [ ]:
weather['zip_code'].value_counts()
# 95113 San Jose
# 94301 Palo Alto
# 94107 San Francisco CA
# 94063 Redwood City
# 94041 Mountain View
# DATOS SACADOS POR GOOGLE MAPS

# Creacion de un dataframe de las cuidades y sus respectivos zip code

In [ ]:
#Creo el dataframe con los zip code de cada cuidad para despues mergear
dato = { 'city' : ['San Jose','Palo Alto','San Francisco','Redwood City','Mountain View'],\
       'zip_code': [95113,94301,94107,94063,94041]}
cityZipCode = pd.DataFrame(dato)
cityZipCode

#### Merge con los otros dataframes

In [ ]:
# Veo cuantos registros tienen en total trips
trips.shape

In [ ]:
trips.head(3)

In [ ]:
#Mergeo asi consigo el zip
stationZip = pd.merge(station,cityZipCode,on='city',how='outer')

In [ ]:
stationZip.head(3)

In [ ]:
# Mergeo con weather asi consigo la ciudad ademas del zip code
weatherCity = pd.merge(weather,cityZipCode,on='zip_code',how='inner')
weatherCity.head(3)

In [ ]:
# Saco las columnas que quiero del stationZip
stationCityZip = stationZip.loc[:,['name','city','zip_code']]
stationCityZip.head(3)

In [ ]:
#Hago dos estaciones uno de salida y otro de entrada, igual que en city y zip code
startStationCityZip = stationCityZip.rename( columns = { 'name'  : 'start_station_name',\
                                                        'city' : 'start_city','zip_code' : 'start_zip_code'})
endStationCityZip = stationCityZip.rename( columns = { 'name'  : 'end_station_name',\
                                                        'city' : 'end_city','zip_code' : 'end_zip_code'})

In [ ]:
startStationCityZip.head(3)

In [ ]:
endStationCityZip.head(3)

#### Se mergea startStationCityZip y endStationCityZip con el trip

In [ ]:
#Ahora hago merge con el trip de la estacion de inicio
tripCityZip = pd.merge(trips,startStationCityZip,on='start_station_name',how='inner')
tripCityZip.head(3)

In [ ]:
#Hago lo mismo que lo anterior mergeo para sacar la ciudad de la estacion final y su zip code
tripCityZip = pd.merge(tripCityZip,endStationCityZip,on='end_station_name',how='inner')
tripCityZip.head(3)

In [ ]:
tripCityZip.shape

#### Al hacer merge del trip con station se ve que desaparecen varios registros debido a que debe haber estaciones que estan en el trip que no aparecen en el station

In [ ]:
estacionesTrip = trips['start_station_name'].value_counts()
estacionesTrip

In [ ]:
estacionesDeStation = station['name'].value_counts()
estacionesDeStation

In [ ]:
indiceStationName = pd.Index(estacionesDeStation.reset_index()['index'])

In [ ]:
indiceStationTripName = pd.Index(estacionesTrip.reset_index()['index'])

#### Estas serian las 4 estaciones que no tienen similitud en los dataset

In [ ]:
indiceStationTripName.difference(indiceStationName)

# Top 10 estacion id de entrada y salida

In [ ]:
#Dame las 10 estaciones que mas veces se usaron como partida
tripCityZip['quantity_start_station_per_id'] = 1

In [ ]:
#Las 10 estaciones de salida mas usadas 
grupo = tripCityZip.groupby('start_station_id').agg('count')\
            .sort_values('id', ascending = False)
topEstacionesSalida = grupo.loc[:,['id']].head(10)
topEstacionesSalida

In [ ]:
#idem con las estaciones de llegada
grupo = tripCityZip.groupby('end_station_id').agg('count')\
            .sort_values('id', ascending = False)
topEstacionesLlegada = grupo.loc[:,['id']].head(10)
topEstacionesLlegada

In [ ]:
plt.title('Top ten de las 10 estaciones que mas llegadas tienen', color = 'Black')
topEstacionesLlegada['id']\
    .plot(kind = 'bar', figsize = (12,8));
plt.xlabel('ID de estacion')
plt.ylabel('Cantidad de bicis que llegaron por estacion');

In [ ]:
plt.title('Top ten de las 10 estaciones que mas salidas tienen', color = 'Black')
topEstacionesSalida['id']\
    .plot(kind = 'bar', figsize = (12,8));
plt.xlabel('ID de estacion') 
plt.ylabel('Cantidad de bicis que salieron por estacion');

# Anio donde hubo mas bicicletas alquiladas/viajes

In [ ]:
tripCityZip['year_start'] = tripCityZip['date_start'].map(lambda x: x.year)
tripCityZip.head(3)

In [ ]:
cantidadPorAnio = tripCityZip['year_start'].value_counts().sort_index()
cantidadPorAnio

In [ ]:
plt.title('Cantidad de bicicletas alquiladas por anios')
cantidadPorAnio.plot('bar',figsize=(14,5))
plt.xlabel('Anios')
plt.ylabel('Cantidad de bicicletas alquiladas');

In [ ]:
#viaje mas largo
tripCityZip.sort_values('duration', ascending = False).head(3)

In [ ]:
weatherCity['events'].value_counts()

In [ ]:
#Rain y rain son lo mismo, lo soluciono
weatherCity['events'] = weatherCity['events'].str.lower()

In [ ]:
weatherCity.groupby('events').count()

In [ ]:
weather_events = weatherCity.dropna(subset = ['events'])

In [ ]:
len(weather_events)

In [ ]:
weather_events.head(3)

In [ ]:
weather_events.loc[weather_events.events.str.contains('rain-thunderstorm'),:]

In [ ]:
weatherCity.loc[weatherCity.date == '02-28-2015']
#para una misma fecha, ocurrieron distintos eventos, tiene sentido pues
#son 5 ciudades distintas, ej: el dia de hoy hubo lluvia en BS AS, pero en
#jujuy pudo haber niebla

# La cuidad que llueve mas

In [ ]:
group_rain = weather_events.groupby('events').get_group('rain')

In [ ]:
#cantidad de dias lluviosos en cada ciudad
group_rain['city'].value_counts()

In [ ]:
plt.figure()
plt.title('Cantidad de dias de lluvia por ciudad', color = 'black')
group_rain.groupby('city').count()['events'].plot(kind = 'bar', figsize=(10,6),\
                                                  color = 'blue');
plt.ylabel('Cantidad de dias')
plt.xlabel('ciudades');

In [ ]:
plt.figure()
plt.title('Cantidad de dias de lluvia por ciudad', color = 'black')
group_rain.groupby('city').count()['events'].plot(kind = 'area', figsize = (10,6), color = 'green');
plt.ylabel('Cantidad de dias')
plt.xlabel('Ciudades');

# Analisis de temperaturas en San Francisco y Palo Alto

In [ ]:
weather_sf = weatherCity.loc[weatherCity.city.str.contains('San Francisco'),:]

In [ ]:
weather_sf.plot.scatter('max_temperature','min_temperature',alpha=0.25,figsize=(10,6))
plt.ylim([0,25])
plt.xlim([0,35]);

In [ ]:
#temperatura en palo alto
weather_pa = weatherCity.loc[weatherCity.city.str.contains('Palo Alto'),:]
weather_pa.plot.scatter('max_temperature','min_temperature',alpha=0.25,figsize=(10,6));
plt.ylim([0,30])
plt.xlim([0,40])

# Merge Trips con Weather

In [ ]:
#Cambio nombre de la columna para poder mergear por la clave
weatherCity = weatherCity.rename(columns={'date' : 'date_start','city': 'start_city'})
weatherToMerge = weatherCity.loc[:,['date_start','max_temperature','mean_temperature','min_temperature','start_city','events']]

In [ ]:
weatherToMerge.head(3)

In [ ]:
tripsWeather = pd.merge(tripCityZip,weatherToMerge,on=['date_start','start_city'],how='inner')

In [ ]:
tripsWeather.head(3)

## Correlacion entre temperatura media y la duracion de viaje

In [ ]:
cor = tripsWeather.loc[:,['duration','mean_temperature']].corr().abs()
cor

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(16,5));
cor = tripsWeather.loc[:,['duration','mean_temperature']].corr().abs()
#cor.values[[np.arange(2)]*2] = 0
sns.heatmap(cor,cmap='Oranges');
# en el heatmap se nota claramente que no hay correlacion entre duracion y temperatura media

## promedio de duracion de viaje en dias de temperatura mayor a 25°

In [ ]:
tripsWeather.loc[tripsWeather.max_temperature >= 25,'duration'].mean()

## promedio de duracion de viaje en dias de temperatura menor a 10°

In [ ]:
tripsWeather.loc[tripsWeather.min_temperature <= 10,'duration'].mean()

### Frecuencia de bicicletas alquiladas para las estaciones que tuvieron mas viajes ( id 70 y 69 )

In [ ]:
frecuenciaID70 = pd.DataFrame()
frecuenciaID69 = pd.DataFrame() 
for statusChunk in pd.read_csv("../input/status.csv", chunksize= 100000):
    estacion70 = statusChunk.loc[statusChunk.station_id == 70,:]
    estacion69 = statusChunk.loc[statusChunk.station_id == 69,:]
    frecuenciaID70 = frecuenciaID70.append(estacion70)
    frecuenciaID69 = frecuenciaID69.append(estacion69)

In [ ]:
# promedio de bicicletas disponibles y docks disponibles para la estacion 69
frecuenciaID69.groupby('station_id').mean()

In [ ]:
# promedio de bicicletas disponibles y docks disponibles para la estacion 70
frecuenciaID70.groupby('station_id').mean()

In [ ]:
plt.figure()
plt.title('Histograma de bicicletas disponibles de las estaciones 69 y 70')
frecuenciaID69['bikes_available'].hist(figsize=(14,5),bins=23,alpha=0.3,color='green',label='Estacion 69')
frecuenciaID70['bikes_available'].hist(figsize=(14,5),bins=20,alpha=0.5,color='blue',label='Estacion 70')
plt.xlabel('Bicicletas disponibles')
plt.ylabel('Cantidad total (medido por cada minuto)')
plt.legend();